In [24]:
# python kernal version 3.10
!pip install pymysql==1.1.0 
!pip install azure-eventhub==5.11.6 

In [25]:
import json
import sqlite3
from sqlite3 import Error
import toml
from azure.eventhub import EventHubConsumerClient

In [26]:
def receive_events_from_event_hub(connection_str, event_hub_name, consumer_group):
    client = EventHubConsumerClient.from_connection_string(
        conn_str=connection_str,
        consumer_group=consumer_group,
        eventhub_name=event_hub_name
    )

    try:
        with client:
            client.receive_batch(
                on_event_batch=lambda events, partition_context: process_events(events, partition_context),
                starting_position="-1",  # Receive events from the beginning
            )
    except Exception as e:
        print(f"Failed to receive events: {e}")

In [27]:
def process_events(events, partition_context):
    # Connect to an SQLite database; adjust as necessary for your database
    try:
        connection = sqlite3.connect('deliveries.db')
        cursor = connection.cursor()
        for event in events:
            # Assuming event data is a JSON string; adjust parsing as necessary
            data = json.loads(event.body_as_str())
            # Insert into the deliveries table; adjust columns as necessary
            sql = "INSERT INTO deliveries (username, pickup_location, delivery_location, date) VALUES (?, ?, ?, ?)"
            cursor.execute(sql, (data['username'], data['pickup_location'], data['delivery_location'], data['date']))
        connection.commit()
    except Error as e:
        print(f"Database error: {e}")
    finally:
        if connection:
            connection.close()

    # Update checkpoint
    partition_context.update_checkpoint()

In [28]:
def read_config_from_toml(file_path):
    """
    Reads the configuration from a TOML file.

    Parameters:
    - file_path: Path to the TOML configuration file.

    Returns:
    A tuple containing the connection string and Event Hub name.
    """
    config = toml.load(file_path)
    return config['event_hub']['connection_string'], config['event_hub']['name']

In [29]:
# Example usage
connection_string, event_hub_name = read_config_from_toml('config.toml')
consumer_group = "$Default"  

receive_events_from_event_hub(connection_string, event_hub_name, consumer_group)

Unable to attach new link: ValueError('Invalid link')
Unable to attach new link: ValueError('Invalid link')
Unable to attach new link: ValueError('Invalid link')
Unable to attach new link: ValueError('Invalid link')


KeyboardInterrupt: 